The data consists of flight arrival and departure details for all commercial flights in the United States from 2003 to 2007. It contains a variety of fields, such as the date, time, airline, origin and destination airports, and information related to departure and arrival delays.

**Import library**
- The import of `os` is used for interacting with the operating system.
- The import of `sqlite3` is used to interact with SQLite database.
- The import of `pandas` is used for data analysis and manipulation
- The import of `numpy` is used for numerical computing in Python.
- The import of `matplotlib.pyplot` allows for plotting.
- The import of `seaborn` is a Python data visualization library based on Matplotlib that provides a high-level interface for creating visually appealing and informative statistical graphics.

In [1]:
import os
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Get current working directory
os.getcwd()

In [2]:
# Set working directory to the location containing the data
os.chdir(r'C:\Working directory')

If the database "flights_python.db" exists in your working directory, the following code chunk will remove it.

In [ ]:
try:
    os.remove('flights_python.db')
except OSError: 
    pass

# Connect database

In [3]:
# Create a connection to the database named 'flights_python.db'
conn = sqlite3.connect('flights_python.db')

# Create tables

First create the tables for the airports, carrier and plane-data datasets 

In [ ]:
airports = pd.read_csv("airports.csv")
carriers = pd.read_csv("carriers.csv")
planes = pd.read_csv("plane-data.csv")

airports.to_sql('airports', con = conn, index = False)
carriers.to_sql('carriers', con = conn, index = False)
planes.to_sql('planes', con = conn, index = False)

Then create the table for flights, which requires several csv files

In [4]:
c = conn.cursor()

In [ ]:
c.execute('''
CREATE TABLE flights (
  Year int,
  Month int,
  DayofMonth int,
  DayOfWeek int,
  DepTime  int,
  CRSDepTime int,
  ArrTime int,
  CRSArrTime int,
  UniqueCarrier varchar(5),
  FlightNum int,
  TailNum varchar(15),
  ActualElapsedTime int,
  CRSElapsedTime int,
  AirTime int,
  ArrDelay int,
  DepDelay int,
  Origin varchar(3),
  Dest varchar(3),
  Distance int,
  TaxiIn int,
  TaxiOut int,
  Cancelled int,
  CancellationCode varchar(1),
  Diverted varchar(1),
  CarrierDelay int,
  WeatherDelay int,
  NASDelay int,
  SecurityDelay int,
  LateAircraftDelay int
)
''')

conn.commit()

Next, we will create a loop to load 2003 to 2007 data files directly from compressed bz2 format and combine them to table named "flights" in the database.

In [ ]:
for year in range(2003, 2008):
    filename = str(year)+".csv.bz2"
    print('Processing:', filename)
    flights = pd.read_csv(filename, encoding="latin-1", low_memory=False, compression='bz2', dtype={"TailNum":"string"}).replace('',pd.NA) #avoids UnicodeDecodeError and mixed data type warning for column 22
    flights.to_sql('flights', con = conn, if_exists = 'append', index = False)

conn.commit()

# Data Cleaning and Preparation

First, we need to ensure that the data is clean and reliable. We will use `pd.read_sql_query` to retrieve the first five rows from the flights table in the database to have a glimpse of the data.

In [ ]:
pd.read_sql_query("SELECT * FROM flights LIMIT 5", conn)

We noticed that there were some missing values in the ‘flights’ data set. Hence, we calculated the percentage of missing values in the data set to give an overview of the data.

`c.execute` execute sql statements.

In [ ]:
# Get column names from the 'flights' table
c.execute("PRAGMA table_info(flights)")
columns = [col[1] for col in c.fetchall()]

# Construct the SQL query to calculate missing value percentages
missing_na_query = f"""
    SELECT {", ".join([
        f"(COUNT(*) - COUNT({col})) * 100.0 / COUNT(*) AS {col}_missing" for col in columns
    ])}
    FROM flights;
"""

# Execute the query
c.execute(missing_na_query)
missing_percentages = c.fetchone()

# Convert to Pandas DataFrame for better readability
df_missing = pd.DataFrame({
    "Column": columns,
    "Missing Percentage": missing_percentages
})

print(df_missing)

We found that “CancellationCode”, “CarrierDelay”, “WeatherDelay”, “NASDelay”, “SecurityDelay”, “LateAircraftDelay” columns have the highest percentage of missing value in the dataset. 

Hence, we created a new table "flights_clean" in the database to avoid modifying the original data. We only kept columns that will be relevant for our analysis. 

In [ ]:
c.execute('''
CREATE TABLE flights_clean AS 
SELECT Year, Month, DayofMonth, DayOfWeek, DepTime, CRSDepTime, ArrTime, CRSArrTime, UniqueCarrier, FlightNum, TailNum, ActualElapsedTime, CRSElapsedTime, AirTime, ArrDelay, DepDelay, Origin, Dest, Distance, Cancelled, Diverted 
From flights ''')

conn.commit()

In [5]:
#List all tables in database
c.execute(''' SELECT name FROM sqlite_master WHERE type='table' ''').fetchall()

[('airports',),
 ('carriers',),
 ('planes',),
 ('flights',),
 ('flights_clean',),
 ('flights_planes',)]

In [ ]:
pd.read_sql_query("SELECT * FROM flights_clean LIMIT 5", conn)

Next, we looked at the missing values in the other columns. 

In [ ]:
c.execute("PRAGMA table_info(flights_clean)")
columns2 = [col[1] for col in c.fetchall()] 

missing_na_query2 = f"""
    SELECT {", ".join([
        f"(COUNT(*) - COUNT({col})) * 100.0 / COUNT(*) AS {col}_missing" for col in columns2
    ])}
    FROM flights_clean;
"""

c.execute(missing_na_query2)
missing_percentages2 = c.fetchone()

df_missing2 = pd.DataFrame({
    "Column": columns2,
    "Missing Percentage": missing_percentages2
})

print(df_missing2)

There were missing values in these columns: DepTime, ArrTime, TailNum, ActualElapsedTime, AirTime, ArrDelay, DepDelay. We then investigated whether the missing values in DepTime or ArrTime were due to flight cancellations or diversions.

In [ ]:
pd.read_sql_query(''' SELECT
    Cancelled,
    Diverted,
    COUNT(*) AS TotalFlights,
    SUM(CASE WHEN DepTime IS NULL THEN 1 ELSE 0 END) AS DepTime_NA,
    SUM(CASE WHEN ArrTime IS NULL THEN 1 ELSE 0 END) AS ArrTime_NA
  FROM flights_clean
  GROUP BY Cancelled, Diverted ''', conn)

We noticed that DepTime and ArrTime were related to the flight either being cancelled or diverted. We can see that when the flights are diverted, there is no ArrTime recorded. Similarly, when the flights are cancelled, there is no DepTime and ArrTime recorded. Therefore, the other columns would also have no values as the flights did not exist. We decided to fill those columns when DepTime or ArrTime is null, the other columns (ActualElapsedTime, CrsElapsedTime, AirTime, ArrDelay, DepDelay) would be 0.

In [ ]:
c.execute(''' 
UPDATE flights_clean
SET DepTime = COALESCE(DepTime, 0),
    ArrTime = COALESCE(ArrTime, 0),
    ActualElapsedTime = COALESCE(ActualElapsedTime, 0),
    CRSElapsedTime = COALESCE(CRSElapsedTime, 0),
    AirTime = COALESCE(AirTime, 0),
    ArrDelay = COALESCE(ArrDelay, 0),
    DepDelay = COALESCE(DepDelay, 0)
WHERE DepTime IS NULL OR ArrTime IS NULL
''')

conn.commit()

In [ ]:
c.execute("PRAGMA table_info(flights_clean)")
columns3 = [col[1] for col in c.fetchall()]

missing_na_query3 = f"""
    SELECT {", ".join([
        f"(COUNT(*) - COUNT({col})) * 100.0 / COUNT(*) AS {col}_missing" for col in columns3
    ])}
    FROM flights_clean;
"""

c.execute(missing_na_query3)
missing_percentages3 = c.fetchone()

df_missing3 = pd.DataFrame({
    "Column": columns3,
    "Missing Percentage": missing_percentages3
})

print(df_missing3)

We noticed that ActualElapsedTime and ArrDelay column still has missing values and went on to investigate.

In [ ]:
pd.read_sql_query("SELECT * FROM flights_clean WHERE ArrDelay IS NULL", conn)

We found that the missing value appeared in the same row. Since only one row had missing values for both columns, manual calculations were performed to determine the correct value. ActualElapsedTime is calculated using DepTime minus CRSDepTime. ArrDelay is calculated using ArrTime minus CRSArrTime.

In [ ]:
c.execute('''
UPDATE flights_clean
SET 
    ActualElapsedTime = CASE 
        WHEN ActualElapsedTime IS NULL THEN 
            CASE 
                WHEN (ArrTime % 100 + (ArrTime / 100) * 60) < (DepTime % 100 + (DepTime / 100) * 60) 
                THEN ((ArrTime % 100 + (ArrTime / 100) * 60) + 1440) - (DepTime % 100 + (DepTime / 100) * 60)
                ELSE (ArrTime % 100 + (ArrTime / 100) * 60) - (DepTime % 100 + (DepTime / 100) * 60)
            END
        ELSE ActualElapsedTime 
    END,
    ArrDelay = CASE 
        WHEN ArrDelay IS NULL THEN 
            CASE 
                WHEN (ArrTime % 100 + (ArrTime / 100) * 60) < (CRSArrTime % 100 + (CRSArrTime / 100) * 60) 
                THEN ((ArrTime % 100 + (ArrTime / 100) * 60) + 1440) - (CRSArrTime % 100 + (CRSArrTime / 100) * 60)
                ELSE (ArrTime % 100 + (ArrTime / 100) * 60) - (CRSArrTime % 100 + (CRSArrTime / 100) * 60)
            END
        ELSE ArrDelay 
    END
WHERE ActualElapsedTime IS NULL OR ArrDelay IS NULL ''')

conn.commit()

In [ ]:
c.execute("PRAGMA table_info(flights_clean)")
columns4 = [col[1] for col in c.fetchall()] 

missing_na_query4 = f"""
    SELECT {", ".join([
        f"(COUNT(*) - COUNT({col})) * 100.0 / COUNT(*) AS {col}_missing" for col in columns4
    ])}
    FROM flights_clean;
"""

c.execute(missing_na_query4)
missing_percentages4 = c.fetchone()

df_missing4 = pd.DataFrame({
    "Column": columns4,
    "Missing Percentage": missing_percentages4
})

print(df_missing4)

We noticed that there is still missing values in TailNum column.

In [ ]:
pd.read_sql_query("SELECT * FROM flights_clean WHERE TailNum IS NULL", conn)

We decided to fill those missing values in TailNum column with 0.

In [ ]:
c.execute(''' 
UPDATE flights_clean
SET TailNum = '0'
WHERE TailNum is NULL
''')

conn.commit()

In [ ]:
c.execute("PRAGMA table_info(flights_clean)")
columns5 = [col[1] for col in c.fetchall()]  

missing_na_query5 = f"""
    SELECT {", ".join([
        f"(COUNT(*) - COUNT({col})) * 100.0 / COUNT(*) AS {col}_missing" for col in columns5
    ])}
    FROM flights_clean;
"""

c.execute(missing_na_query5)
missing_percentages5 = c.fetchone()

df_missing5 = pd.DataFrame({
    "Column": columns5,
    "Missing Percentage": missing_percentages5
})

print(df_missing5)

Now, there are no missing values in the flights data set. We will proceed to prepare the data for analysis.

We will first change DepTime and ArrTime for those timing that are more than 2400hrs. In the DepTime and ArrTime columns, some values exceeded 2400. To ensure correct time conversion, a conditional statement was applied: if a time value exceeded 2400, 2400 was subtracted to obtain the correct time.

In [ ]:
c.execute(''' UPDATE flights_clean
  SET
    DepTime = CASE
      WHEN DepTime >= 2400 THEN DepTime - 2400
      ELSE DepTime
    END,
    ArrTime = CASE
      WHEN ArrTime >= 2400 THEN ArrTime - 2400
      ELSE ArrTime
    END
  WHERE DepTime >= 2400 OR ArrTime >= 2400''')

conn.commit()

We also converted the DayOfWeek column from numeric values to their corresponding day names.

In [ ]:
c.execute('''UPDATE flights_clean
SET DayOfWeek = CASE 
WHEN DayOfWeek = 1 THEN 'Monday'
WHEN DayOfWeek = 2 THEN 'Tuesday'
WHEN DayOfWeek = 3 THEN 'Wednesday'
WHEN DayOfWeek = 4 THEN 'Thursday'
WHEN DayOfWeek = 5 THEN 'Friday'
WHEN DayOfWeek = 6 THEN 'Saturday'
WHEN DayOfWeek = 7 THEN 'Sunday'
END''')

conn.commit()

## (a) What are the best times and days of the week to minimise delays each year?

### (i) Best day of the week

In this section, we will examine the percentage and average of flight delay by Day of Week.

#### Plot 1: Percentage of Flight Delay by Day of Week for Each Year

We will be using arrival delay as the primary indicator of delay. If the flight arrives more than 15 minutes, it is considered delayed.

In [ ]:
percent_delay = pd.read_sql_query(''' SELECT Year, DayOfWeek, 
                        (SUM(CASE WHEN ArrDelay > 15 THEN 1 END) * 100.0 / COUNT(*)) AS DelayPercentage
                        FROM flights_clean
                        GROUP BY Year, DayOfWeek ''', conn)

In [ ]:
day_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
percent_delay["DayOfWeek"] = pd.Categorical(percent_delay["DayOfWeek"], categories=day_order, ordered=True)

We will plot a bar plot using `sns.barplot()` function, where the x-axis represent the year and y-axis shows the percentage of flights delayed. The bars are grouped by the day of the week, allowing for easy comparison across different days. `plt.figure` sets the overall figure size to make the plot more readable. A `for` loop iterates over each bar in the plot and adds labels displaying the percentage of flight delays. The labels are placed near the top of each bar using the `annotate()` function. The plot is further customized with a title, axis labels and legend for clarity. `plt.tight_layout()` ensures that elements are well-spaced and do not overlap and `plt.show()` displays the plot.

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=percent_delay, x="Year", y="DelayPercentage", hue="DayOfWeek", dodge=True)

for p in plt.gca().patches:
    height = p.get_height()
    if height > 0:  # Avoid placing labels on empty bars
        plt.gca().annotate(f"{height:.1f}", 
                           (p.get_x() + p.get_width() / 2., height-1), 
                           ha='center', va='center', 
                           fontsize=10, fontweight='bold', 
                           rotation=90, color="black")

plt.title("Figure 3.2 (Python): Percentage of Flight Delay by Day of Week for Each Year", fontsize=12, fontweight="bold")
plt.xlabel("Year", fontsize=10)
plt.ylabel("Percentage of Flight Delay (%)", fontsize=10)
plt.legend(title="Day of Week", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=0) 

plt.tight_layout()
plt.show()

**Observations from the plot above**

There is an increasing trend in flight delays over the years, with notable fluctuations on different days of the week. We can see that delays tend to be more frequent on certain days such as Thursday and Friday across all years. Friday consistently recorded the highest percentage of delays from 2003 to 2007 with an increase from 17.5% to 26.7%. Meanwhile, Saturday consistently recorded the lowest percentage of delays from 2003 to 2007 with an increase from 12.6% to 19.5%. This pattern suggests that travellers should anticipate longer delays towards the end of the work week, especially on Thursday and Friday. 

#### Plot 2: Average Flight Delay by Day of Week

In [ ]:
avg_delay = pd.read_sql_query(''' SELECT Year, DayOfWeek, AVG(ArrDelay) AS AvgDelay
                                    FROM flights_clean
                                    WHERE Cancelled = 0 AND Diverted = 0 AND ArrDelay > 15
                                    GROUP BY Year, DayOfWeek ''', conn)

In [ ]:
day_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
avg_delay["DayOfWeek"] = pd.Categorical(avg_delay["DayOfWeek"], categories=day_order, ordered=True)

We will plot a bar plot using `sns.barplot()` function, where the x-axis represent the year and y-axis shows the average flight delay. The bars are grouped by the day of the week. Similarly, a `for` loop iterates over each bar in the plot and adds labels displaying the average duration of flight delay. The labels are placed near the top of each bar using the `annotate()` function. The plot is further customized with a title, axis labels and legend.

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=avg_delay, x="Year", y="AvgDelay", hue="DayOfWeek", dodge=True)

for p in plt.gca().patches:
    height = p.get_height()
    if height > 0: 
        plt.gca().annotate(f"{height:.1f}", 
                           (p.get_x() + p.get_width() / 2., height-2), 
                           ha='center', va='center', 
                           fontsize=10, fontweight='bold', 
                           rotation=90, color="black")

plt.title("Figure 4.2 (Python): Average Flight Delay by Day of Week for Each Year", fontsize=12, fontweight="bold")
plt.xlabel("Year", fontsize=10)
plt.ylabel("Average Flight Delay (minutes)", fontsize=10)
plt.legend(title="Day of Week", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=0)  

plt.tight_layout()
plt.show()

**Observations from the plot above**

Saturday has the lowest average delay from 2003 to 2007 with an increase in duration from 47.7 minutes to 53.9 minutes. This suggests a decline in flights punctuality over the years with flight delays becoming more frequent and longer. Despite this increasing trend, Saturday remains the best day with the lowest delays for 5 consecutive years. Therefore, the best day to minimize flight delays is on Saturday.

### (ii) Best time

To determine the best time to minimize delays, we created the TimeInterval column by grouping arrival times into eight three-hour intervals. This allows us to analyze delay patterns across different times of the day.

In [ ]:
c.execute("ALTER TABLE flights_clean ADD COLUMN TimeInterval TEXT")

In [ ]:
c.execute(''' UPDATE flights_clean
    SET TimeInterval = 
        CASE 
            WHEN ArrTime BETWEEN 0 AND 259 THEN '0000-0259'
            WHEN ArrTime BETWEEN 300 AND 559 THEN '0300-0559'
            WHEN ArrTime BETWEEN 600 AND 859 THEN '0600-0859'
            WHEN ArrTime BETWEEN 900 AND 1159 THEN '0900-1159'
            WHEN ArrTime BETWEEN 1200 AND 1459 THEN '1200-1459'
            WHEN ArrTime BETWEEN 1500 AND 1759 THEN '1500-1759'
            WHEN ArrTime BETWEEN 1800 AND 2059 THEN '1800-2059'
            WHEN ArrTime BETWEEN 2100 AND 2359 THEN '2100-2359'
        END ''')

conn.commit()

In this section, we will examine the percentage and average of flight delay by Time Interval.

#### Plot 3: Percentage of Flight Delay by Time Interval

In [ ]:
percent_time_delay = pd.read_sql_query(''' SELECT Year, TimeInterval, 
                        (SUM(CASE WHEN ArrDelay > 15 THEN 1 END) * 100.0 / COUNT(*)) AS DelayPercentage
                        FROM flights_clean
                        WHERE Cancelled = 0 AND Diverted = 0
                        GROUP BY Year, TimeInterval''', conn)

We will plot a line graph using `sns.lineplot()` function, where the x-axis represent the time interval and y-axis shows the percentage of flights delayed. The `hue="Year"` argument assigns different colors to each year, making it easy to compare trends across years. The plot is customized with a title, axis labels and legend. To enhance readability, `plt.grid()` adds a dashed grid, making it easier to compare values across different time intervals.

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(data=percent_time_delay, x="TimeInterval", y="DelayPercentage", hue="Year", 
             marker="o", palette="Dark2", linewidth=1.5)

plt.title("Figure 5.2 (Python) : Percentage of Flight Delay by Time Interval for Each Year", fontsize=14, fontweight="bold")
plt.xlabel("Time Interval", fontsize=12)
plt.ylabel("Percentage of Flight Delay (%)", fontsize=12)
plt.legend(title="Year")

plt.grid(True, linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()

##### Lowest Percentage of Delayed Flights by Time Interval for Each Year

In [ ]:
lowest_percent_delay = pd.read_sql_query('''
    WITH ranked_percent_time_delays AS (
        SELECT Year, TimeInterval,
        (SUM(CASE WHEN ArrDelay > 15 THEN 1 END) * 100.0 / COUNT(*)) AS DelayPercentage,
        ROW_NUMBER() OVER (PARTITION BY Year ORDER BY (SUM(CASE WHEN ArrDelay > 15 THEN 1 END) * 100.0 / COUNT(*))) AS rank
        FROM flights_clean
        WHERE Cancelled = 0 AND Diverted = 0
        GROUP BY Year, TimeInterval
    )
    SELECT Year, TimeInterval, DelayPercentage
    FROM ranked_percent_time_delays
    WHERE rank = 1
''', conn)

pd.DataFrame(lowest_percent_delay)

**Observations from the plot and table above**

Early morning hours from 0600 to 0859 consistently show the lowest percentage of flight delay from 2003 to 2007 with an increase from 5.2% to 8.2%.  

#### Plot 4: Average Flight Delay by Time Interval

In [ ]:
avg_time_delay = pd.read_sql_query(''' SELECT Year, TimeInterval, AVG(ArrDelay) as AvgDelay
                        FROM flights_clean
                        WHERE ArrDelay > 15 AND Cancelled = 0 AND Diverted = 0
                        GROUP BY Year, TimeInterval ''', conn)

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(data=avg_time_delay, x="TimeInterval", y="AvgDelay", hue="Year", 
             marker="o", palette="Dark2", linewidth=1.5)

plt.title("Figure 6.2 (Python): Average Flight Delay by Time Interval for Each Year", fontsize=14, fontweight="bold")
plt.xlabel("Time Interval", fontsize=12)
plt.ylabel("Average Flight Delay (minutes)", fontsize=12)
plt.legend(title="Year")

plt.grid(True, linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()

##### Lowest Average Flight Delay by Time Interval for Each Year

In [ ]:
lowest_avg_delay = pd.read_sql_query('''
    WITH ranked_time_delays AS (
        SELECT Year, TimeInterval, AVG(ArrDelay) AS AvgDelay,
               ROW_NUMBER() OVER (PARTITION BY Year ORDER BY AVG(ArrDelay)) AS rank
        FROM flights_clean
        WHERE Cancelled = 0 AND Diverted = 0 AND ArrDelay > 15
        GROUP BY Year, TimeInterval
    )
    SELECT Year, TimeInterval, AvgDelay
    FROM ranked_time_delays
    WHERE rank = 1
''', conn)

pd.DataFrame(lowest_avg_delay)

**Observations from the plot and table above**

We observe that flight delays are generally highest in the early morning hours especially during 0300 to 0559. This indicates that flights during this period experience significantly longer delays than other times of the day. Following this peak, there is a sharp drop in average delay during the 0600 to 0859, where delays reach their lowest point from 2003 to 2007 as indicated by the table above, which shows the lowest average flight delay for each year. However, the average flight delay during this interval has increased from 35.2 minutes to 37.3 minutes over the years. Despite this slight rise, early mornings between 0600 and 0859 remain the best option for minimizing delays. 

## Conclusion for (a)

Saturday and early mornings from 0600 to 0859 are the best day and times to minimise flight delays. Travelers seeking to reduce the likelihood of delays should prioritize early morning flights and consider scheduling their trips on Saturdays. By focusing on these optimal times and days, airlines and passengers alike can enhance punctuality and improve overall travel efficiency. 

## (b) Evaluate whether older planes suffer more delays on a year-to-year basis. 


Before merging the flights and planes datasets, we first rename the 'year' column in the planes dataset to avoid confusion, as both datasets contain a column with the same name.

In [ ]:
c.execute('''ALTER TABLE planes RENAME COLUMN year to manu_year''')

conn.commit()

In [ ]:
c.execute('''UPDATE planes SET manu_year = CAST(manu_year as INTEGER)''')

conn.commit()

We will create a table in the database named "flights_planes" where we will join the "flights_clean" and "planes" table. 

In [ ]:
c.execute('''CREATE TABLE flights_planes AS SELECT Year, Month, DayofMonth, DayOfWeek, flights_clean.TailNum, ArrDelay, manu_year 
            From flights_clean
            JOIN planes ON flights_clean.TailNum = planes.tailnum
            WHERE Cancelled = 0 AND Diverted = 0''')

conn.commit()

Next, we create a new column 'PlanesAge' to determine the planes age.

In [ ]:
c.execute('''ALTER TABLE flights_planes ADD COLUMN PlanesAge INTEGER''')
conn.commit()

Planes Age is calculated using Year minus manu_year, which is the manufactured year of plane.

In [ ]:
c.execute('''UPDATE flights_planes 
                SET PlanesAge = Year - manu_year''')
conn.commit()

We noticed that the manu_year for some planes is 0 and it results in the incorrect calculation of the planes age. Therefore, we decided to rows where manu_year is 0 as it could lead to inaccuracy for our analysis.

In [ ]:
c.execute('''DELETE FROM flights_planes WHERE manu_year = 0''')
conn.commit()

We also noticed that the PlanesAge for some planes is less than 0. Hence, we decided to remove these rows as planes age cannot be less than 0. 

In [ ]:
c.execute('''DELETE FROM flights_planes WHERE PlanesAge < 0 ''')
conn.commit()

### Plot 5: Distribution of Number of Flights by Planes Age

We will first find the distribution of the number of flights by Planes Age to understand how many flights each age group would take.

In [ ]:
age_distribution = pd.read_sql_query(''' SELECT PlanesAge, COUNT(*) / 1000000.0 AS FlightCount
                           FROM flights_planes
                           GROUP BY PlanesAge''', conn)

We will plot a histogram using `plt.hist`. The `age_distribution['PlanesAge']` data represents the age of the planes, and the `bins=np.arange(...)` argument divides the data into intervals of one year, from the minimum to the maximum age of the planes. The plot is then customized with a title, axis labels and legend.

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(age_distribution['PlanesAge'], bins=np.arange(age_distribution['PlanesAge'].min(), 
                                                        age_distribution['PlanesAge'].max() + 1, 1), 
         weights=age_distribution['FlightCount'], color='skyblue', edgecolor='black')

plt.title("Figure 7.2 (Python): Distribution of Number of Flights by Planes Age", weight = 'bold')
plt.xlabel('Planes Age (Year)')
plt.ylabel('Number of Flights (millions)')

plt.show()

**Observations from the plot above**

We can see that planes older than 22 years have significantly fewer flights compared to planes younger than 22 years old. Due to this smaller sample size, the average delay of planes aged more than 22 may not accurately represent the true average delay. To address this, we will analyze planes aged between 0 and 22 and planes aged more than 22 to better understand the relationship between plane age and flight delays. 

### Plot 6: Percentage of Delayed flights by Planes Age (0 to 22 years)

In [ ]:
delay_percent_by_age = pd.read_sql_query(''' SELECT PlanesAge,
                                            (SUM(CASE WHEN ArrDelay > 15 THEN 1 END) * 100.0 / COUNT(*)) AS DelayPercentage
                                            FROM flights_planes
                                            WHERE PlanesAge <= 22
                                            GROUP BY PlanesAge ''', conn)

We will plot a scatter plot with a regression line to explore the relationship between the age of planes and the percentage of flight delays. The `sns.jointplot()` creates the plot, where the x-axis represent the plane age and the y-axis shows the percentage of flight delayed. `kind="reg"` indicates that a regression line should be added to the scatter plot, which helps to visualize the correlation between plane age and percentage of flight delay.

In [ ]:
plot_6 = sns.jointplot(
        data=delay_percent_by_age, 
        x="PlanesAge", 
        y="DelayPercentage", 
        kind="reg",  
    )

plot_6.set_axis_labels('Planes Age (Year)', 'Percentage of Flight Delay (%)')
plot_6.fig.suptitle("Figure 8.2 (Python): Percentage Delay based on Age of Plane (0 to 22)", y = 1.02, fontsize=10, weight='bold')

plt.show()

**Observations from the plot above**

There is a positive correlation indicating that as planes get older, the percentage of delayed flights increases. Although the trend is upward-sloping, individual data points exhibit noticeable variations

### Plot 7: Percentage of Delayed flights by Planes Age (more than 22)

In [ ]:
delay_percent_by_age2 = pd.read_sql_query('''SELECT PlanesAge,
        (SUM(CASE WHEN ArrDelay > 15 THEN 1 END) * 100.0 / COUNT(*)) AS DelayPercentage
        FROM flights_planes
        WHERE PlanesAge > 22
        GROUP BY PlanesAge ''', conn)

In [ ]:
plot_7 = sns.jointplot(
        data=delay_percent_by_age2, 
        x="PlanesAge", 
        y="DelayPercentage", 
        kind="reg",  
    )

plot_7.set_axis_labels('Planes Age (Year)', 'Percentage of Flight Delay (%)')
plot_7.fig.suptitle("Figure 9.2 (Python): Percentage Delay based on Planes Age (more than 22)", y = 1.02, fontsize=10, weight='bold')

plt.show()

**Observations from the plot above**

There is a weak positive correlation between plane age and percentage of delays. This suggest that older planes aged over 22 years may experience a small increase in delay percentage, but the effect is not strong. The data points are widely scattered around the trend line, suggests that older planes do not necessarily experience higher delays at a consistent rate.

### Plot 8: Average Flight Delay based on Planes Age (0 to 22)

In [ ]:
avg_delay_by_age = pd.read_sql_query('''SELECT PlanesAge, AVG(ArrDelay) AS AvgDelay
                                        FROM flights_planes
                                        WHERE PlanesAge <= 22 AND ArrDelay > 15
                                        GROUP BY PlanesAge''', conn)

In [ ]:
plot_8 = sns.jointplot(
    data=avg_delay_by_age, 
    x="PlanesAge", 
    y="AvgDelay", 
    kind="reg",  
)

plot_8.set_axis_labels('Planes Age (Year)', 'Average Flight Delay (minutes)')
plot_8.fig.suptitle("Figure 10.2 (Python): Average Flight Delay based on Plane Age (0 to 22)", fontsize=10, weight='bold')

plt.tight_layout()
plt.show()

**Observations from the plot above**

There is a negative correlation between average flight delay and planes age between 0 and 22. The regression line slopes downward suggest that as planes age within this range, delays tend to decrease.  

### Plot 9: Average Flight Delay based on Planes Age (more than 22)

In [ ]:
avg_delay_by_age2 = pd.read_sql_query(''' SELECT PlanesAge, AVG(ArrDelay) AS AvgDelay
                                        FROM flights_planes
                                        WHERE PlanesAge > 22 AND ArrDelay > 15
                                        GROUP BY PlanesAge''', conn)


In [ ]:
plot_9 = sns.jointplot(
    data=avg_delay_by_age2, 
    x="PlanesAge", 
    y="AvgDelay", 
    kind="reg", 
)

plot_9.set_axis_labels('Planes Age (Year)', 'Average Flight Delay (minutes)')
plot_9.fig.suptitle(" Figure 11.2 (Python): Average Flight Delay based on Planes Age (more than 22)", y = 1.02, fontsize=10, weight='bold')
plt.show()

**Observations from the plot above**

There is a positive correlation between planes age and flight delay. The regression line slopes upwards, indicating that as planes get older, they tend to experience longer delays. However, the relationship is weak as indicated by the scattered data points and the wide confidence interval around the regression line.

Overall, these findings suggest that average delays decrease as planes age from 0 and 22 years but begin to increase for planes older than 22 years. However, the trend for planes older than 22 years is less reliable due to the smaller sample size. The data for planes aged 0 to 22 years is more representative and reliable, making it the focus of our yearly trend analysis.

### Plot 10: Yearly Trend of Percentage Delay based on Planes Age (0 to 22) (Representative)

In [ ]:
yearly_percent_delay = pd.read_sql_query(''' SELECT Year, PlanesAge, 
                               (SUM(CASE WHEN ArrDelay > 15 THEN 1 END) * 100.0 / COUNT(*)) AS DelayPercentage
                               FROM flights_planes
                               WHERE PlanesAge <= 22
                               GROUP BY Year, PlanesAge ''', conn)

We will create a series of scatter plots with regression lines to show how the percentage of flight delays changes with plane age over different years. The `sns.FacetGrid()` is used to create multiple subplots, one for each year. The `col="Year` specifies that the plots will be separated by year, and the `col_wrap=3` ensures that a maximum of three plots are displayed per row, wrapping to the next row as needed.The `.map()` is then used to create scatter plots for each year, where the x-axis represents the plane's age and the y-axis represents the percentage of flight delays. The `plt.scatter` creates the scatter plots, and the `add_legend()` adds a legend to each subplot. `.map_dataframe()` adds a red regression line to each scatter plot, with `sns.regplot()`.

In [ ]:
plot_10 = sns.FacetGrid(yearly_percent_delay, col="Year", col_wrap =3)
plot_10.map(plt.scatter, "PlanesAge", "DelayPercentage").add_legend()
plot_10.map_dataframe(sns.regplot, x="PlanesAge", y="DelayPercentage", scatter=False, color="red")

plot_10.set_axis_labels("Plane Age (Year)", "Percentage of Flight Delay (%)")

plt.subplots_adjust(top=0.9)
plot_10.fig.suptitle("Figure 12.2 (Python): Yearly Trend of Percentage Delay based on Planes Age (0 to 22)", fontsize=12, fontweight="bold")
plt.show()

**Observations from the plot above**

In 2003, 2004 and 2007, there is a positive correlation between plane age and percentage of delayed flights. This suggests that in these years, older planes generally experienced a higher percentage of delays compared to younger planes. In 2005, the trend appears to be flat, indicating little relationship between plane age and percentage delay. This suggest that planes age did not impact the delay. In 2006, there is a negative correlation. This suggests that newer planes had a slightly higher percentage of delays compared to older planes.

### Plot 11: Yearly Trend of Average Delay based on Planes Age (0 to 22) (Representative)

In [ ]:
yearly_delay = pd.read_sql_query(''' SELECT Year, PlanesAge, AVG(ArrDelay) AS AvgDelay
                               FROM flights_planes
                               WHERE ArrDelay > 15 AND PlanesAge <= 22
                               GROUP BY Year, PlanesAge ''', conn)


In [ ]:
plot_11 = sns.FacetGrid(yearly_delay, col="Year", col_wrap =3)
plot_11.map(plt.scatter, "PlanesAge", "AvgDelay").add_legend()
plot_11.map_dataframe(sns.regplot, x="PlanesAge", y="AvgDelay", scatter=False, color="red")

plot_11.set_axis_labels("Plane Age (Year)", "Average Flight Delay (minutes)")

plt.subplots_adjust(top=0.9)
plot_11.fig.suptitle("Figure 13.2 (Python): Yearly Trend of Average Delay based on Planes Age (0 to 22)", fontsize=12, fontweight="bold")
plt.show()

**Observations from the plot above**

In 2003, 2004, and 2005, there is a positive correlation suggests that older planes tend to have slightly higher average flight delays. The confidence intervals in 2004 and 2005 are slightly wider, indicating some uncertainty in the trend. In 2006 and 2007, there is a negative correlation, where older planes exhibit shorter average delays compared to newer ones. 

## Conclusion for (b)

While older planes aged more than 22 experience increasing delays over time, there is a weak correlation between plane age and delays. For planes aged between 0 to 22, the percentage of delayed flights increases with age, but their average delay time does not necessarily worsen. While there is some evidence that plane age contributes to delays, the effect is not consistent across all years, indicating planes age is not the only determining factor for flight delays.

## (c) Logistic regression model

In this section, the aim is to fit a logistic regression model for the probability of diverted US flights from 2003 to 2007 based on various flight features such as scheduled departure and arrival times, distance and carrier. To better understand these trends, we will visualize the regression coefficients across the years.

To build the model, we use `sklearn` package in Python as these packages are efficient tools for machine learning.

- "sklearn.model_selection import train_test_split": divides the dataset into a training set and a test set    
- "sklearn.linear_model import LogisticRegression": imports the LogisticRegression model, which is used for binary or multi-class classification.
- "sklearn.pipeline import Pipeline": allows chaining multiple preprocessing and modeling steps into a single workflow for efficiency and consistency. 
- "sklearn.preprocessing import StandardScaler, OneHotEncoder": StandardScaler standardizes numerical features by scaling them while OneHotEncoder convertes categorical features into a format suitable for machine learning models
- "sklearn.compose import ColumnTransformer": transform different types

In [ ]:
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer 

We will join flights, airports and carriers table together and select specific features that are relevant for our model.

In [ ]:
flights_merged = pd.read_sql_query('''
                    SELECT f.Year, f.Month, f.DayofMonth, f.DayOfWeek, f.CRSDepTime, f.CRSArrTime, f.Distance, f.UniqueCarrier, 
                    a1.lat As OriginLat, a1.long AS OriginLong, a2.lat AS DestLat, a2.long AS DestLong, f.Diverted
                    FROM flights_clean f
                    JOIN airports a1 ON f.Origin = a1.iata
                    JOIN airports a2 ON f.Dest = a2.iata
                    WHERE f.Cancelled = 0 ''', conn)

In [ ]:
flights_merged.info()

We will count the number of diverted and non-diverted flights to understand the distribution of diverted flights in the dataset. To count the number of diverted and non-diverted flight, we need to convert the Diverted column from object data type to numeric data type first.

In [ ]:
flights_merged.dtypes

In [ ]:
flights_merged['Diverted'] = flights_merged['Diverted'].astype(int)

In [ ]:
diverted_count = (flights_merged["Diverted"] == 1).sum()
non_diverted_count = (flights_merged["Diverted"] == 0).sum()

print(diverted_count)
print(non_diverted_count)

There are 72558 diverted flights and 3463546 non-diverted flights. As the sample size for non-diverted flights are larger than the diverted flight, we decrease the size of the majority class to be the same to the minority class.

In [ ]:
np.random.seed(123)

# Count the number of diverted flights
diverted_count = flights_merged[flights_merged["Diverted"] == 1].shape[0]

# Filter diverted flights (Diverted == 1)
diverted_flights = flights_merged[flights_merged["Diverted"] == 1]

# Downsample non-diverted flights
non_diverted_flights = flights_merged[flights_merged["Diverted"] == 0].sample(n=diverted_count, random_state=123)

# Combine both datasets
flights_sampled = pd.concat([diverted_flights, non_diverted_flights], ignore_index=True)


We will then train the logistic regression model for each year. It involves data preparation, preprocessing, model training and extracting coefficients to analyze how different features influence flight diversions over time.

The features are separated in three different categories: numerical, categorical and target.

In [ ]:
numerical_features = ['Month', 'DayofMonth', 'CRSDepTime', 'CRSArrTime','Distance','OriginLat','OriginLong', 'DestLat', 'DestLong']
categorical_features = ['DayOfWeek', 'UniqueCarrier']
target = 'Diverted'

An empty dictionary is created to store the coefficients.

In [ ]:
coefficients = {}

We first loop through each year in the data to ensure that a separate logistic regression model is trained for each year. Within the loop, it extracts only the rows for the current year. X selects the features needed for the model while Y selects the target variable, Diverted, which indicates whether a flight was diverted or not. Next, we will set up the pre-processing steps for numerical and categorical features.

For numerical feature, we apply scaling with the function `StandardScaler()`, which standardizes the data. For categorical feature, we apply OneHotEncoder() function, which converts categorical values into binary columns. We combine these transformations using the function `ColumnTransformer()`, specifying the transformations with the `transformers` argument.  This creates a new pipeline called `data_transformer`.

We are now ready to complete our machine learning pipeline by simply attaching a learner to the `data_transformer` pipeline.

The next stage is to perform the standard machine learning operations of training the pipeline and evaluating its predictive performance. We begin by splitting the data into train and test datasets with the very convenient function `train_test_split()`. The data set is divided into 50% training data and 50% testing data.

After that, the model learns from the training data, figuring out which features are important for predicting whether a flight is diverted.

Once the model is trained, it retrieves the feature names after preprocessing and stores the model's coefficients for the current year. It then repeats the process for all years.

In [ ]:
for year in flights_sampled['Year'].unique():
    flights_sampled_year = flights_sampled[flights_sampled['Year'] == year]
    X = flights_sampled_year[numerical_features + categorical_features]
    y = flights_sampled_year[target]
    
    data_transformer = ColumnTransformer(
        transformers=[
        ('numerical', StandardScaler(), numerical_features),
        ('categorical', OneHotEncoder(handle_unknown='ignore', drop=None), categorical_features)]) 
    
    model = Pipeline(steps=[('data_transformer', data_transformer),
                            ('model', LogisticRegression(max_iter=1000))])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)
    
    model.fit(X_train, y_train)

    feature_names = (numerical_features + list(model.named_steps['data_transformer'].named_transformers_['categorical']
                                         .get_feature_names_out(categorical_features)))

    coefficients[year] = model.named_steps['model'].coef_.flatten()


In [ ]:
# Convert to DataFrame
coefs_df = pd.DataFrame.from_dict(coefficients, orient='index', columns = feature_names)

### Plot 12: Logistic regression coefficients

We will plot a heatmap to visualize the coefficients of the logistic regression model.

The coefficients are first categorized into different color ranges:
- Coefficients less than or equal to -1 are represented by red
- Coefficient between -1 and 0 are represented by pink
- Coefficient between 0 and 1 are represented by blue
- Coefficient greater than 1 are represented by green.

In [ ]:
coefs_colored = coefs_df.T.copy()
coefs_colored[coefs_colored <= -1] = -1 #red
coefs_colored[(coefs_colored > -1) & (coefs_colored <= 0)] = -0.5 #pink
coefs_colored[(coefs_colored > 0) & (coefs_colored <= 1)] = 0.5 #blue
coefs_colored[coefs_colored > 1] = 1 #green

`sns.heatmap()` is used to generate the heatmap, with the `cmap` argument specifying the colors for the different coefficient ranges, the `center=0` argument ensuring that 0 is centered on the color scale, and `linewidths=0.5` with a black line color around each cell to improve the visual clarity of the heatmap.

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(coefs_df.T, cmap=["red", "pink", "lightblue", "green"], center=0, linewidths=0.5, linecolor = 'black')

plt.xlabel("Year")
plt.ylabel("Feature")
plt.title("Logistic Regression Coefficients", weight = 'bold')
plt.show()

**Observations from the plot above**

Carrier HA exhibit strong negative coefficients in 2007, highlighted in red. This suggests that flights operated by this airline were significantly less likely to be diverted during those periods. On the other hand, carrier XE show more positive coefficients in 2006, highlighted in green. This indicates that flights operated by this carrier were more likely to be diverted during that year. Other features such as distance and time generally have more consistent positive or negative effects across years.

### Conclusion for (c)

Factors such as time and airport locations appear to have less impact on the probabilities of flight diversion while airline carriers seem to have some impact on the probabilities of flight diversion.

## Close the connection

We can close the connection using the method `close` on `conn`:

In [6]:
conn.close()